In [3]:
import warnings
warnings.filterwarnings('ignore')
import os

import tws_data 
import statsmodels.tsa.stattools as ts
import statsmodels.api as sm
from ib_insync import *
import numpy as np
import pandas as pd
import datetime as dt
import time
pd.options.display.float_format = '{:,.4f}'.format
%matplotlib inline
from scipy.stats import norm as norm
import matplotlib.pyplot as plt
import requests as req
from bs4 import BeautifulSoup as bs
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import math


In [28]:
os.chdir('D:\Options Data\IB Intraday')

def create_dfrow(curr_option):
    df_row = pd.DataFrame({'Symbol': curr_option.contract.localSymbol, 
                           'Type':curr_option.contract.right, 
                           'Bid': curr_option.bid, 
                           'Ask': curr_option.ask, 
                           'Volume': curr_option.volume, 
                           'Strike': curr_option.contract.strike,
                           'Expiry': dt.datetime.strptime(curr_option.contract.lastTradeDateOrContractMonth, '%Y%m%d'),
                           'bidIV': curr_option.bidGreeks.impliedVol, 
                           'bidDelta': curr_option.bidGreeks.delta,
                           'bidGamma': curr_option.bidGreeks.gamma, 
                           'bidVega': curr_option.bidGreeks.vega,
                           'bidTheta': curr_option.bidGreeks.theta, 
                           'askIV': curr_option.askGreeks.impliedVol, 
                           'askDelta': curr_option.askGreeks.delta,
                           'askGamma': curr_option.askGreeks.gamma, 
                           'askVega': curr_option.askGreeks.vega,
                           'askTheta': curr_option.askGreeks.theta,
                           'modelIV': curr_option.modelGreeks.impliedVol, 
                           'modelDelta': curr_option.modelGreeks.delta,
                           'modelGamma': curr_option.modelGreeks.gamma, 
                           'modelVega': curr_option.modelGreeks.vega,
                           'modelTheta': curr_option.modelGreeks.theta}, index = [0])
    return df_row

def time_remaining(weekly_options, curr_spx):
    typ = weekly_options.reset_index().loc[0,'Type']

    try:
        interest_rate = bs(req.get('https://www.marketwatch.com/investing/bond/tmubmusd01m?countrycode=bx','lxml').text, 'lxml')
        interest_rate = float(interest_rate.select_one('h3[class*="intraday__price"]').select_one('bg-quote').text)/100
    except:
        interest_rate = 0.02422

    weekly_options['a'] = interest_rate - (weekly_options['modelIV']**2)/2

    if typ == 'P':
        weekly_options['b'] = weekly_options['modelIV']*norm.ppf(-weekly_options['modelDelta'])
    if typ == 'C':
        weekly_options['b'] = -weekly_options['modelIV']*norm.ppf(weekly_options['modelDelta'])
    weekly_options['c'] = np.log(curr_spx/weekly_options['Strike'])

    weekly_options_otm = weekly_options[abs(weekly_options.modelDelta) <= 0.5]
    weekly_options_itm = weekly_options[abs(weekly_options.modelDelta) > 0.5]

    if typ == 'P':
        weekly_options_otm['time_remaining'] = ((-weekly_options_otm.b - 
                                                 np.sqrt(weekly_options_otm.b**2 - 
                                                         4*weekly_options_otm.a*weekly_options_otm.c))/(2*weekly_options_otm.a))**2
        weekly_options_itm['time_remaining'] = ((-weekly_options_itm.b + 
                                                 np.sqrt(weekly_options_itm.b**2 - 
                                                         4*weekly_options_itm.a*weekly_options_itm.c))/(2*weekly_options_itm.a))**2
    if typ == 'C':
        weekly_options_otm['time_remaining'] = ((-weekly_options_otm.b + 
                                                 np.sqrt(weekly_options_otm.b**2 - 
                                                         4*weekly_options_otm.a*weekly_options_otm.c))/(2*weekly_options_otm.a))**2
        weekly_options_itm['time_remaining'] = ((-weekly_options_itm.b - 
                                                 np.sqrt(weekly_options_itm.b**2 - 
                                                         4*weekly_options_itm.a*weekly_options_itm.c))/(2*weekly_options_itm.a))**2
    weekly_times = pd.concat([weekly_options_otm, 
                              weekly_options_itm], axis = 0)[['Expiry','time_remaining']].dropna().groupby('Expiry').mean()

    del weekly_options['a'], weekly_options['b'], weekly_options['c'], weekly_options_itm, weekly_options_otm

    return weekly_options.merge(weekly_times.reset_index(), on = 'Expiry')

def get_options(ticker, exchange, contractType, earliest_date, latest_date, curr_iv, stdevs, save_csv = False):
    if contractType == 'Stock':
        security = Stock(ticker,exchange)
    else:
        security = Index(ticker, exchange)

    ib.qualifyContracts(security)

    chains = ib.reqSecDefOptParams(security.symbol, '', security.secType, security.conId)

#     chains_df = util.df(chains)
#     chain = next(c for c in chains if c.tradingClass == ticker and c.exchange == 'SMART')
    chain = list(filter(lambda c: c.tradingClass == 'SPXW' and c.exchange == 'SMART', chains))[0]

    all_expirations = sorted(exp for exp in chain.expirations)
    expirations_dates = [dt.datetime.strptime(x, '%Y%m%d') for x in all_expirations]

    expirations = list(filter(lambda x: x[0] >= earliest_date and x[0] <= latest_date, list(zip(expirations_dates,all_expirations))))
    dte_expirations = [(expir[0] - dt.datetime.today()).days for expir in expirations]
    expirations = [expir[1] for expir in expirations]

    ib.reqHeadTimeStamp(security, whatToShow='TRADES', useRTH=True)

    end_date = '' #'20100506 13:00:00 PST'
    duration = '1 D'

    sec_bars = ib.reqHistoricalData(
            security,
            endDateTime=end_date,
            durationStr=duration,
            barSizeSetting='1 min',
            whatToShow='TRADES',
            useRTH=True,
            formatDate=1)


    sec_df = util.df(sec_bars).set_index('date')
    sec_df.index = pd.to_datetime(sec_df.index)

    curr_sec_price = sec_df.reset_index().loc[len(sec_df) - 1, 'close']
    bounds_lst = [curr_sec_price*(stdevs*stdevs/(100*np.sqrt(365.25/dte))) for dte in dte_expirations]


    contracts_list = []

    for bounds, expiration in zip(bounds_lst, expirations):
#         strikes = [strike for strike in chain.strikes
#                    if strike % 5 == 0
#                    and curr_sec_price - bounds < strike < curr_sec_price + bounds]
        strikes = [strike for strike in chain.strikes
                   if curr_sec_price - bounds < strike < curr_sec_price + bounds]

        rights = ['P', 'C']

        curr_contracts = [Option(ticker, expiration, strike, right, 'SMART')
                          for right in rights
                          for strike in strikes]
        contracts_list += [curr_contracts]

    puts_list = []
    calls_list = []

    for contracts in contracts_list:
        ib.qualifyContracts(*contracts)
        sec_options = ib.reqTickers(*contracts)

        puts_rows = []
        calls_rows = []

        for curr_option in sec_options:
            try:
                curr_row = create_dfrow(curr_option)
                if curr_option.contract.right == 'P':
                    puts_rows.append(curr_row)
                elif curr_option.contract.right == 'C':
                    calls_rows.append(curr_row)
            except:
                continue

        puts = pd.concat(puts_rows).reset_index(drop = True).sort_values(['Expiry','Strike'])
        calls = pd.concat(calls_rows).reset_index(drop = True).sort_values(['Expiry','Strike'])
        puts = time_remaining(puts, curr_sec_price)
        calls = time_remaining(calls, curr_sec_price)
        puts_list.append(puts)
        calls_list.append(calls)
        time.sleep(5)

    puts_df = pd.concat(puts_list,axis = 0).reset_index(drop = True)
    calls_df = pd.concat(calls_list,axis = 0).reset_index(drop = True)
    
    puts_df['Mid'] = (puts_df['Ask'] + puts_df['Bid'])/2
    calls_df['Mid'] = (calls_df['Ask'] + calls_df['Bid'])/2
    if save_csv:
        curr_time = dt.datetime.now().strftime('%Y%m%d-%H-%M')
        puts_df.to_csv('ib_puts' + curr_time + '.csv')
        calls_df.to_csv('ib_calls' + curr_time + '.csv')
        
    return puts_df, calls_df, curr_sec_price

def get_spreads(weekly_options, curr_spx, expiry_index = 0, rowshift = 1, commissions = 0.0266):

    expirations_dates = weekly_options.Expiry.drop_duplicates().tolist()

    weekly_options = weekly_options[weekly_options.Expiry == expirations_dates[expiry_index]]

    typ = weekly_options.reset_index().loc[0,'Type']

    if typ == 'P':
        weekly_options = weekly_options.sort_values('Strike', ascending = False).reset_index(drop = True)
    if typ == 'C':
        weekly_options = weekly_options.sort_values('Strike', ascending = True).reset_index(drop = True)

    time_remaining = weekly_options.reset_index(drop = True).loc[0,'time_remaining']

    weekly_options['Mid'] = (weekly_options.Ask + weekly_options.Bid)/2
    shorts = weekly_options[['Mid', 'Strike', 'bidDelta', 'bidGamma', 'bidTheta', 'bidVega']]
    shorts.columns = ['Price','Strike','Delta','Gamma','Theta','Vega']
    longs = weekly_options[['Mid','Strike', 'askDelta', 'askGamma','askTheta', 'askVega']]
    longs.columns = ['Price','Strike','Delta','Gamma','Theta','Vega']

    spreads = shorts - longs.shift(-rowshift)
    spreads['Short_Strike'] = shorts.Strike
    spreads['Short_Delta'] = shorts.Delta
    spreads['Long_Strike'] = longs.Strike.shift(-rowshift)
    spreads = spreads[['Short_Strike', 'Long_Strike', 'Short_Delta', 'Price',
                       'Strike', 'Delta', 'Gamma', 'Theta', 'Vega']]

    spreads['Credit'] = spreads.Price - commissions

    if typ == 'P':
        spreads['MaxLoss'] = (-spreads.Strike + spreads.Credit)*100
        spreads['BreakEven'] = spreads['Short_Strike'] - spreads.Credit
    if typ == 'C':
        spreads['MaxLoss'] = (spreads.Strike + spreads.Credit)*100
        spreads['BreakEven'] = spreads['Short_Strike'] + spreads.Credit

    del spreads['Price'], spreads['Strike']

    spreads = spreads[spreads.Credit > 0].dropna().reset_index(drop = True)
    spreads['Expiry'] = expirations_dates[expiry_index]

    return spreads

def art_hedges(curr_calls, curr_puts, curr_price, expiry_index, bias = 'upside',
               maxshift = 6, commissions = 0.0266):
    
    hedge = []
    for rowshift in range(1,maxshift):

        if bias == 'upside':
            spreads = get_spreads(curr_puts, curr_price, expiry_index, rowshift, commissions = 0.0266)
        else:
            spreads = get_spreads(curr_calls, curr_price, expiry_index, rowshift, commissions = 0.0266)
        spreads = spreads[(spreads.Credit > 0.5)]
        hedge.append(spreads)

    hedge = pd.concat(hedge, axis = 0).reset_index(drop = True)

    if bias == 'upside':
        expirations_dates = curr_calls.Expiry.drop_duplicates().tolist()

        long_hedge = curr_calls[(curr_calls.Expiry == expirations_dates[expiry_index])][['Ask', 'Bid', 'Expiry', 'Strike',
                                                                             'modelDelta', 'modelGamma',
                                                                             'modelTheta', 'modelVega', 'time_remaining']].reset_index(drop = True)
    else:
        expirations_dates = curr_puts.Expiry.drop_duplicates().tolist()

        long_hedge = curr_puts[(curr_puts.Expiry == expirations_dates[expiry_index])][['Ask', 'Bid', 'Expiry', 'Strike',
                                                                             'modelDelta', 'modelGamma',
                                                                             'modelTheta', 'modelVega', 'time_remaining']].reset_index(drop = True)

    long_hedge['Mid'] = (long_hedge['Ask'] + long_hedge['Bid'])/2
    long_hedge.columns = [x.replace('model','') for x in long_hedge.columns.tolist()]

    hedges_lst = []

    for idx, row in hedge.iterrows():

        curr_hedge = long_hedge[long_hedge.Mid < row.Credit][['Mid','Expiry', 'Strike',
                                                              'Delta', 'Gamma',
                                                              'Theta', 'Vega', 'time_remaining']]
        curr_hedge['Gamma'] = curr_hedge['Gamma'] - row.Gamma
        curr_hedge['Delta'] = curr_hedge['Delta'] - row.Delta
        curr_hedge['Theta'] = curr_hedge['Theta'] - row.Theta
        curr_hedge['Vega'] = curr_hedge['Vega'] - row.Vega
        if bias == 'upside':
            curr_hedge = curr_hedge[(curr_hedge.Delta > 0) &
                                    (curr_hedge.Gamma > 0) &
                                    (curr_hedge.Theta > 0) &
                                    (curr_hedge.Vega > 0)]
        else:
            curr_hedge = curr_hedge[(curr_hedge.Delta < 0) &
                                (curr_hedge.Gamma > 0) &
                                (curr_hedge.Theta > 0) &
                                (curr_hedge.Vega > 0)]
        curr_hedge['Credit'] = -curr_hedge.Mid + row.Credit - 0.0133

        if bias == 'upside':
            curr_hedge['MaxLoss'] = (row.Long_Strike - row.Short_Strike + curr_hedge.Credit)*100
            curr_hedge['BreakEven'] = row.Short_Strike - curr_hedge['Credit']
        else:
            curr_hedge['MaxLoss'] = (row.Short_Strike - row.Long_Strike + curr_hedge.Credit)*100
            curr_hedge['BreakEven'] = row.Short_Strike + curr_hedge['Credit']
        curr_hedge['Maturity'] = curr_hedge['Expiry']
        curr_hedge['Mid'] = row.Short_Strike
        curr_hedge['Expiry'] = row.Long_Strike

        if bias == 'upside':
            curr_hedge.columns = ['Short_Put_Strike','Long_Put_Strike','Long_Call_Strike'] + curr_hedge.columns.tolist()[3:]
        else:
            curr_hedge.columns = ['Short_Call_Strike','Long_Call_Strike','Long_Put_Strike'] + curr_hedge.columns.tolist()[3:]

        hedges_lst.append(curr_hedge)

    hedges_df = pd.concat(hedges_lst, axis = 0)
    
    return hedges_df

def get_spreads2(curr_puts, curr_calls, maxshift):
    puts_sum = curr_puts[['Mid', 'Strike'] + list(filter(lambda x: 'model' in x, curr_puts.columns))].sort_values('Strike').reset_index(drop = True)
    calls_sum = curr_calls[['Mid', 'Strike'] + list(filter(lambda x: 'model' in x, curr_calls.columns))].sort_values('Strike').reset_index(drop = True)
    
    def get_spreads_temp(curr_opt_df, opt_type, maxshift):
        spreads_df = []
        for i in range(1,maxshift + 1):
            if opt_type == 'p':
                spread_sum = curr_opt_df - curr_opt_df.shift(-i).reset_index(drop = True)
                spread_sum = spread_sum.join(curr_opt_df[['Strike']], rsuffix = '_long_p').join(curr_opt_df.shift(-i)[['Strike']], rsuffix = '_short_p').dropna()
            elif opt_type == 'c':
                spread_sum = curr_opt_df - curr_opt_df.shift(i).reset_index(drop = True)
                spread_sum = spread_sum.join(curr_opt_df[['Strike']], rsuffix = '_long_c').join(curr_opt_df.shift(i)[['Strike']], rsuffix = '_short_c').dropna()

            spread_sum = spread_sum[list(filter(lambda x: x != 'Strike', spread_sum.columns))]
            spread_sum = spread_sum.rename(columns = {"Mid": "Price"}) 
            spreads_df.append(spread_sum)

        spreads_df = pd.concat(spreads_df, axis = 0).reset_index(drop = True)
        return spreads_df
    
    call_spreads = get_spreads_temp(calls_sum, 'c', maxshift)
    put_spreads = get_spreads_temp(puts_sum, 'p', maxshift)
    
    return put_spreads, call_spreads

def spreads_multiplier(spreads, max_multipliers = 5):
    curr_spreads_df = []

    for i in range(1, max_multipliers + 1):
        curr_spreads = spreads.copy()
        curr_spreads.iloc[:,:6] = curr_spreads.iloc[:,:6]*i
        curr_spreads['SpreadMultiplier'] = i
        curr_spreads_df.append(curr_spreads)

    curr_spreads_df = pd.concat(curr_spreads_df, axis = 0).reset_index(drop = True)
    return curr_spreads_df

In [5]:
client_id = 11

ib = IB()
ib.connect('127.0.0.1', 7496, clientId=client_id)

<IB connected to 127.0.0.1:7496 clientId=11>

In [6]:
exchange = 'CBOE'
contractType = 'Index'
earliest_date = dt.datetime(2020,6,23)
latest_date = dt.datetime(2020,7,1)

ticker = 'SPX'
curr_iv = 20
stdevs = 20
curr_puts, curr_calls, curr_price = get_options(ticker, exchange, contractType, earliest_date, latest_date, curr_iv, stdevs, save_csv = False)
curr_puts = curr_puts[curr_puts.Bid > 0]
curr_calls = curr_calls[curr_calls.Bid > 0]

Started to throttle requests
Error 200, reqId 9: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200630', strike=2560.0, right='P', exchange='SMART')
Error 200, reqId 13: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200630', strike=2565.0, right='P', exchange='SMART')
Error 200, reqId 14: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200630', strike=3080.0, right='P', exchange='SMART')
Error 200, reqId 15: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200630', strike=2570.0, right='P', exchange='SMART')
Error 200, reqId 17: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200630', strike=3085.0, right='P', exchange='SMART')
Error 200, r

Error 200, reqId 71: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200630', strike=100.0, right='P', exchange='SMART')
Error 200, reqId 70: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200630', strike=3170.0, right='P', exchange='SMART')
Error 200, reqId 72: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200630', strike=2660.0, right='P', exchange='SMART')
Error 200, reqId 76: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200630', strike=2665.0, right='P', exchange='SMART')
Error 200, reqId 77: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200630', strike=3180.0, right='P', exchange='SMART')
Error 200, reqId 78: No security definiti

Error 200, reqId 130: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200630', strike=700.0, right='P', exchange='SMART')
Error 200, reqId 133: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200630', strike=3260.0, right='P', exchange='SMART')
Error 200, reqId 134: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200630', strike=2240.0, right='P', exchange='SMART')
Error 200, reqId 135: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200630', strike=2755.0, right='P', exchange='SMART')
Error 200, reqId 136: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200630', strike=2245.0, right='P', exchange='SMART')
Error 200, reqId 138: No security de

Error 200, reqId 191: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200630', strike=2335.0, right='P', exchange='SMART')
Error 200, reqId 192: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200630', strike=800.0, right='P', exchange='SMART')
Error 200, reqId 195: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200630', strike=2340.0, right='P', exchange='SMART')
Error 200, reqId 196: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200630', strike=2855.0, right='P', exchange='SMART')
Error 200, reqId 197: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200630', strike=2345.0, right='P', exchange='SMART')
Error 200, reqId 198: No security de

Error 200, reqId 252: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200630', strike=1930.0, right='P', exchange='SMART')
Error 200, reqId 254: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200630', strike=2445.0, right='P', exchange='SMART')
Error 200, reqId 255: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200630', strike=400.0, right='P', exchange='SMART')
Error 200, reqId 256: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200630', strike=1425.0, right='P', exchange='SMART')
Error 200, reqId 258: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200630', strike=2960.0, right='P', exchange='SMART')
Error 200, reqId 259: No security de

...

Error 200, reqId 989: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200630', strike=2605.0, right='C', exchange='SMART')
Error 200, reqId 990: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200630', strike=3120.0, right='C', exchange='SMART')
Error 200, reqId 991: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200630', strike=2610.0, right='C', exchange='SMART')
Error 200, reqId 994: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200630', strike=2615.0, right='C', exchange='SMART')
Error 200, reqId 995: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200630', strike=3130.0, right='C', exchange='SMART')
Error 200, reqId 996: No security d

Error 200, reqId 1050: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200630', strike=3210.0, right='C', exchange='SMART')
Error 200, reqId 1051: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200630', strike=2190.0, right='C', exchange='SMART')
Error 200, reqId 1052: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200630', strike=3215.0, right='C', exchange='SMART')
Error 200, reqId 1053: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200630', strike=2705.0, right='C', exchange='SMART')
Error 200, reqId 1054: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200630', strike=3220.0, right='C', exchange='SMART')
Error 200, reqId 1055: No secu

Error 200, reqId 1107: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200630', strike=1250.0, right='C', exchange='SMART')
Error 200, reqId 1110: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200630', strike=2790.0, right='C', exchange='SMART')
Error 200, reqId 1111: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200630', strike=2280.0, right='C', exchange='SMART')
Error 200, reqId 1112: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200630', strike=2795.0, right='C', exchange='SMART')
Error 200, reqId 1113: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200630', strike=2285.0, right='C', exchange='SMART')
Error 200, reqId 1114: No secu

Error 200, reqId 1170: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200630', strike=2895.0, right='C', exchange='SMART')
Error 200, reqId 1171: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200630', strike=2385.0, right='C', exchange='SMART')
Error 200, reqId 1172: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200630', strike=850.0, right='C', exchange='SMART')
Error 200, reqId 1175: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200630', strike=2390.0, right='C', exchange='SMART')
Error 200, reqId 1176: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200630', strike=2905.0, right='C', exchange='SMART')
Error 200, reqId 1177: No secur

Error 200, reqId 1230: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200630', strike=2995.0, right='C', exchange='SMART')
Error 200, reqId 1231: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200630', strike=2485.0, right='C', exchange='SMART')
Error 200, reqId 1232: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200630', strike=950.0, right='C', exchange='SMART')
Error 200, reqId 1235: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200630', strike=2490.0, right='C', exchange='SMART')
Error 200, reqId 1236: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20200630', strike=1980.0, right='C', exchange='SMART')
Error 200, reqId 1237: No secur

In [29]:

maxshift = 30
commissions = 0.0266
put_spreads, call_spreads = get_spreads2(curr_puts, curr_calls, maxshift)

put_spreads = put_spreads[put_spreads.Strike_short_p <= curr_price].sort_values(['Strike_short_p','Strike_long_p']).reset_index(drop = True)
call_spreads = call_spreads[call_spreads.Strike_short_c >= curr_price].sort_values(['Strike_short_c','Strike_long_c']).reset_index(drop = True)

ps_mult = spreads_multiplier(put_spreads, max_multipliers = 1)
cs_mult = spreads_multiplier(call_spreads, max_multipliers = 1)

puts_sum = curr_puts[['Mid', 'Strike'] + list(filter(lambda x: 'model' in x, curr_puts.columns))].sort_values('Strike').reset_index(drop = True)
calls_sum = curr_calls[['Mid', 'Strike'] + list(filter(lambda x: 'model' in x, curr_calls.columns))].sort_values('Strike').reset_index(drop = True)


In [41]:
ps_mult.head()

Price  modelDelta  modelGamma  modelIV  modelTheta  modelVega  \
0 -0.3750      0.0012     -0.0000   0.0197      0.0037    -0.0890   
1 -0.6250      0.0021     -0.0000   0.0282      0.0062    -0.1492   
2 -0.2500      0.0009     -0.0000   0.0085      0.0025    -0.0602   
3 -0.9250      0.0031     -0.0000   0.0368      0.0089    -0.2152   
4 -0.5500      0.0018     -0.0000   0.0170      0.0052    -0.1262   

   Strike_long_p  Strike_short_p  SpreadMultiplier  
0     1,200.0000      1,300.0000                 1  
1     1,200.0000      1,350.0000                 1  
2     1,300.0000      1,350.0000                 1  
3     1,200.0000      1,400.0000                 1  
4     1,300.0000      1,400.0000                 1

In [40]:
# Upside Check
art_upsides = []

for idx, row in calls_sum[calls_sum.Mid <= abs(ps_mult.Price.min())].iterrows():
    curr_art_pos = ps_mult[['Strike_short_p']]
    curr_art_pos['Strike_long_p'] = ps_mult['Strike_long_p']
    curr_art_pos['SpreadMultiplier'] = ps_mult['SpreadMultiplier']
    curr_art_pos['Strike_long_c'] = row.Strike
    curr_art_pos['Credit'] = ps_mult['Price'] + row.Mid - ps_mult['SpreadMultiplier']*commissions
    curr_art_pos['modelDelta'] = ps_mult['modelDelta'] + row.modelDelta
    curr_art_pos['modelGamma'] = ps_mult['modelGamma'] + row.modelGamma
    curr_art_pos['modelVega'] = ps_mult['modelVega'] + row.modelVega
    curr_art_pos['modelTheta'] = ps_mult['modelTheta'] + row.modelTheta
    curr_art_pos['MaxLoss'] = ((curr_art_pos['Strike_long_p'] - curr_art_pos['Strike_short_p'])*ps_mult['SpreadMultiplier'] - curr_art_pos['Credit'])*100
    art_upsides.append(curr_art_pos)
    
# Downside Check
art_downsides = []

for idx, row in puts_sum[puts_sum.Mid <= abs(cs_mult.Price.min())].iterrows():
    curr_art_pos = cs_mult[['Strike_short_c']]
    curr_art_pos['Strike_long_c'] = cs_mult['Strike_long_c']
    curr_art_pos['SpreadMultiplier'] = cs_mult['SpreadMultiplier']
    curr_art_pos['Strike_long_p'] = row.Strike
    curr_art_pos['Credit'] = cs_mult['Price'] + row.Mid - cs_mult['SpreadMultiplier']*commissions
    curr_art_pos['modelDelta'] = cs_mult['modelDelta'] + row.modelDelta
    curr_art_pos['modelGamma'] = cs_mult['modelGamma'] + row.modelGamma
    curr_art_pos['modelVega'] = cs_mult['modelVega'] + row.modelVega
    curr_art_pos['modelTheta'] = cs_mult['modelTheta'] + row.modelTheta
    curr_art_pos['MaxLoss'] = ((curr_art_pos['Strike_short_c'] - curr_art_pos['Strike_long_c'])*cs_mult['SpreadMultiplier'] - curr_art_pos['Credit'])*100
    art_downsides.append(curr_art_pos)
    
art_upsides = pd.concat(art_upsides, axis = 0).reset_index(drop = True)
art_downsides = pd.concat(art_downsides, axis = 0).reset_index(drop = True)


In [42]:
potential_ups = art_upsides[(art_upsides.Credit < 0) &
                            (art_upsides.modelGamma > 0) &
                            (art_upsides.modelVega > 0) &
                            (art_upsides.modelTheta > 0)].sort_values('Credit').reset_index(drop = True)

potential_downs = art_downsides[(art_downsides.Credit < 0) &
                                (art_downsides.modelGamma > 0) &
                                (art_downsides.modelVega > 0) &
                                (art_downsides.modelTheta > 0)].sort_values('Credit').reset_index(drop = True)



potential_ups['ReturnOnRisk'] = abs(potential_ups.Credit/potential_ups.MaxLoss)
potential_downs['ReturnOnRisk'] = abs(potential_downs.Credit/potential_downs.MaxLoss)

In [43]:
try:
    potential_ups['downmove_to_breach'] = potential_ups['Strike_short_p']/curr_price - 1
    potential_ups['upmove_to_win'] = potential_ups['Strike_long_c']/curr_price - 1
    potential_ups = potential_ups.sort_values('MaxLoss').reset_index(drop = True)
    potential_ups['MaxLossScore'] = potential_ups.index
    
    potential_ups = potential_ups.sort_values('downmove_to_breach', ascending = False).reset_index(drop = True)
    potential_ups['DownMoveScore'] = potential_ups.index
    
    potential_ups = potential_ups.sort_values('upmove_to_win', ascending = False).reset_index(drop = True)
    potential_ups['UpMoveScore'] = potential_ups.index
    
    potential_ups['Score'] = potential_ups.MaxLossScore + potential_ups.DownMoveScore + potential_ups.UpMoveScore
    potential_ups = potential_ups.sort_values('Score', ascending = False)
except:
    None

In [44]:
potential_ups[potential_ups.Credit < -0.5]

Strike_short_p  Strike_long_p  SpreadMultiplier  Strike_long_c   Credit  \
141       1,525.0000     1,400.0000                 1     3,900.0000  -0.5266   
129       1,600.0000     1,500.0000                 1     3,900.0000  -0.6516   
117       1,625.0000     1,525.0000                 1     3,900.0000  -0.7766   
200       1,700.0000     1,575.0000                 1     3,800.0000  -1.1766   
134       1,575.0000     1,450.0000                 1     3,900.0000  -0.7266   
72        1,550.0000     1,450.0000                 1     4,000.0000  -0.6016   
76        1,575.0000     1,475.0000                 1     4,000.0000  -0.7516   
120       1,650.0000     1,550.0000                 1     3,900.0000  -0.9016   
214       1,725.0000     1,600.0000                 1     3,800.0000  -1.4266   
305       1,725.0000     1,550.0000                 1     3,700.0000  -1.5016   
458       1,800.0000     1,575.0000                 1     3,600.0000  -2.0516   
211       1,650.0000     1,500.0000                 1     3,800.0000  -1.0766   
44        1,625.0000     1,550.0000                 1     4,000.0000  -0.7266   
216       1,675.0000     1,525.0000                 1     3,800.0000  -1.3016   
133       1,600.0000     1,475.0000                 1     3,900.0000  -0.8766   
293       1,775.0000     1,625.0000                 1     3,700.0000  -1.6516   
17        1,575.0000     1,500.0000                 1     4,100.0000  -0.6516   
166       1,675.0000     1,550.0000                 1     3,800.0000  -1.0266   
21        1,600.0000     1,525.0000                 1     4,100.0000  -0.7266   
36        1,600.0000     1,525.0000                 1     4,000.0000  -0.6016   
30        1,650.0000     1,600.0000                 1     4,100.0000  -0.5516   
34        1,500.0000     1,400.0000                 1     4,100.0000  -0.6016   
1         1,550.0000     1,475.0000                 1     4,100.0000  -0.5266   
65        1,475.0000     1,350.0000                 1     4,000.0000  -0.5516   
4         1,625.0000     1,575.0000                 1     4,100.0000  -0.5016   
33        1,675.0000     1,625.0000                 1     4,100.0000  -0.6516   
130       1,550.0000     1,400.0000                 1     3,900.0000  -0.8016   
112       1,675.0000     1,575.0000                 1     3,900.0000  -1.0516   
121       1,625.0000     1,500.0000                 1     3,900.0000  -1.0516   
48        1,650.0000     1,575.0000                 1     4,000.0000  -0.7766   
...              ...            ...               ...            ...      ...   
1180      2,375.0000     1,700.0000                 1     3,325.0000 -26.8766   
1446      2,475.0000     1,725.0000                 1     3,275.0000 -33.2766   
1406      2,575.0000     1,850.0000                 1     3,275.0000 -47.5266   
1402      2,525.0000     1,800.0000                 1     3,275.0000 -39.6766   
1448      2,475.0000     1,725.0000                 1     3,250.0000 -28.3766   
690       2,050.0000     1,300.0000                 1     3,450.0000 -11.1766   
1399      2,550.0000     1,825.0000                 1     3,275.0000 -43.4766   
1450      2,525.0000     1,775.0000                 1     3,250.0000 -35.5766   
1050      2,350.0000     1,700.0000                 1     3,350.0000 -26.5766   
1449      2,500.0000     1,750.0000                 1     3,250.0000 -31.8766   
1187      2,375.0000     1,675.0000                 1     3,325.0000 -27.3766   
701       2,050.0000     1,200.0000                 1     3,450.0000 -11.5516   
1323      2,425.0000     1,700.0000                 1     3,300.0000 -30.3266   
1204      2,350.0000     1,625.0000                 1     3,325.0000 -25.1266   
1082      2,325.0000     1,625.0000                 1     3,350.0000 -24.9766   
1026      2,275.0000     1,575.0000                 1     3,350.0000 -20.0766   
1336      2,475.0000     1,750.0000                 1     3,300.0000 -36.8266   
1411      2,550.0000     1,800.0000  

In [48]:
potential_downs[(potential_downs.Strike_short_c > 3025) &
                (potential_downs.MaxLoss > -6000)]

Strike_short_c  Strike_long_c  SpreadMultiplier  Strike_long_p   Credit  \
467      3,050.0000     3,125.0000                 1     1,600.0000 -28.3266   
469      3,050.0000     3,125.0000                 1     1,625.0000 -27.9266   
481      3,075.0000     3,150.0000                 1     1,650.0000 -25.1266   
500      3,050.0000     3,100.0000                 1     1,450.0000 -20.0516   

     modelDelta  modelGamma  modelVega  modelTheta     MaxLoss  ReturnOnRisk  
467     -0.0958      0.0000     0.0206      0.0039 -4,667.3400        0.0061  
469     -0.0970      0.0000     0.0950      0.0011 -4,707.3400        0.0059  
481     -0.0987      0.0000     0.0006      0.0005 -4,987.3400        0.0050  
500     -0.0626      0.0000     0.0334      0.0016 -2,994.8400        0.0067

In [207]:
curr_puts[curr_puts.Strike == 2805]

Ask    Bid     Expiry     Strike                 Symbol Type   Volume  \
90 0.2000 0.1000 2019-06-24 2,805.0000  SPXW  190624P02805000    P 388.0000   

    askDelta  askGamma  askIV  ...    bidIV  bidTheta bidVega modelDelta  \
90   -0.0064    0.0003 0.2300  ...   0.2120   -0.0972  0.0285    -0.0054   

   modelGamma modelIV modelTheta  modelVega  time_remaining    Mid  
90     0.0003  0.2246    -0.1060     0.0422          0.0083 0.1500  

[1 rows x 24 columns]

In [205]:
ps_mult[(ps_mult.Strike_short_p == 2805) & 
        (ps_mult.SpreadMultiplier == 5)]

Price  modelDelta  modelGamma  modelIV  modelTheta  modelVega  \
2985 -1.3750      0.0802     -0.0055   0.4225      0.9896    -0.4870   
2986 -1.2500      0.0745     -0.0048   0.3838      0.9782    -0.4554   
2987 -1.0000      0.0578     -0.0038   0.3646      0.7341    -0.3607   
2988 -1.0000      0.0480     -0.0032   0.3378      0.6120    -0.3040   
2989 -0.7500      0.0409     -0.0027   0.3085      0.5307    -0.2618   
2990 -0.7500      0.0349     -0.0022   0.2789      0.4611    -0.2252   
2991 -0.7500      0.0305     -0.0019   0.2474      0.4170    -0.1981   
2992 -0.5000      0.0255     -0.0016   0.2188      0.3548    -0.1672   
2993 -0.5000      0.0201     -0.0012   0.1934      0.2773    -0.1332   
2994 -0.5000      0.0160     -0.0010   0.1662      0.2212    -0.1067   
2995 -0.3750      0.0136     -0.0008   0.1341      0.2000    -0.0915   
2996 -0.2500      0.0098     -0.0006   0.1091      0.1410    -0.0662   
2997 -0.2500      0.0051     -0.0003   0.0903      0.0578    -0.0346   
2998 -0.2500      0.0030     -0.0002   0.0613      0.0315    -0.0204   
2999 -0.1250      0.0015     -0.0001   0.0304      0.0165    -0.0102   

      Strike_short_p  Strike_long_p  SpreadMultiplier  
2985      2,805.0000     2,730.0000                 5  
2986      2,805.0000     2,735.0000                 5  
2987      2,805.0000     2,740.0000                 5  
2988      2,805.0000     2,745.0000                 5  
2989      2,805.0000     2,750.0000                 5  
2990      2,805.0000     2,755.0000                 5  
2991      2,805.0000     2,760.0000                 5  
2992      2,805.0000     2,765.0000                 5  
2993      2,805.0000     2,770.0000                 5  
2994      2,805.0000     2,775.0000                 5  
2995      2,805.0000     2,780.0000                 5  
2996      2,805.0000     2,785.0000                 5  
2997      2,805.0000     2,790.0000                 5  
2998      2,805.0000     2,795.0000                 5  
2999      2,805.0000     2,800.0000                 5

In [50]:
potential_downs = arteHedges_downside[(arteHedges_downside.Short_Call_Strike > curr_price)].sort_values(['Maturity', 
                                                                                                         'Long_Put_Strike'], ascending = False)
try:
    potential_downs['upmove_to_breach'] = potential_downs['Short_Call_Strike']/curr_price - 1
    potential_downs['downmove_to_win'] = potential_downs['Long_Put_Strike']/curr_price - 1
    potential_downs = potential_downs.sort_values('MaxLoss').reset_index(drop = True)
    potential_downs['MaxLossScore'] = potential_downs.index
    
    potential_downs = potential_downs.sort_values('upmove_to_breach').reset_index(drop = True)
    potential_downs['UpMoveScore'] = potential_downs.index
    
    potential_downs = potential_downs.sort_values('downmove_to_win').reset_index(drop = True)
    potential_downs['DownMoveScore'] = potential_downs.index
    
    potential_downs['Score'] = potential_downs.MaxLossScore + potential_downs.UpMoveScore + potential_downs.DownMoveScore
    potential_downs = potential_downs.sort_values('Score', ascending = False)
except:
    None
potential_downs

Short_Call_Strike  Long_Call_Strike  Long_Put_Strike   Delta  Gamma  Theta  \
1         2,955.0000        3,005.0000       2,950.0000 -0.8904 0.0005 0.0079   
0         2,955.0000        3,010.0000       2,950.0000 -0.8913 0.0004 0.0065   

    Vega  time_remaining  Credit     MaxLoss  BreakEven   Maturity  \
1 0.0853          0.0085  0.0601 -4,993.9900 2,955.0601 2019-06-24   
0 0.0796          0.0085  0.0601 -5,493.9900 2,955.0601 2019-06-24   

   ReturnOnRisk  upmove_to_breach  downmove_to_win  MaxLossScore  UpMoveScore  \
1        0.0000            0.0008          -0.0009             1            1   
0        0.0000            0.0008          -0.0009             0            0   

   DownMoveScore  Score  
1              1      3  
0              0      0